# Importar librerías / import the libraries

In [1]:
import json
import os
import pandas as pd

# Leer los datos desde la carpeta json / read data from json folder

Los json guardan los keypoints detectados con openpose

the json files save the keypoint from openpose

In [ ]:
json_folder1 = './output/new_bible_out'

#json_folders = [json_folder1, json_folder2]

In [ ]:
x_train = []
y_train = []
file_name = []

In [ ]:
for sub in os.listdir(json_folder1):
    for file in sorted(os.listdir(json_folder1 + '/' + sub)):
        for x in sorted(os.listdir(json_folder1 + '/' + sub + '/' + file)):
            with open(json_folder1 + '/' + sub + '/' + file + '/' + x) as f:
                data = json.load(f)
                try:
                    pose = data['people'][0]['pose_keypoints_2d']
                    handL = data['people'][0]['hand_left_keypoints_2d']
                    handR = data['people'][0]['hand_right_keypoints_2d']
                except IndexError:
                    continue
                del pose[2::3]
                del handL[2::3]
                del handR[2::3]
                final = pose[2:14] + handL + handR
                x_train.append(pose[2:14] + handL + handR)
                y_train.append(file)
                file_name.append(sub + '_' + file + '_' + x.split('_')[0] + '.mp4')

# Convertir a dataframe / convert it into dataframe

In [ ]:
df1 = pd.DataFrame(x_train)
df1 = ['label'] = y_train
df1['filename'] = file_name

In [ ]:
df1.to_csv('./output/test_new.csv')

In [ ]:
df1.head(50)

In [ ]:
#get value count
file_names = df1['filename'].value_counts()
labels = df1['label'].value_counts().index
labels

# Fijar 12 frames por video / set the frames to 12 for each video file

In [ ]:
final_csv = pd.DataFrame()
for index, value in file_names.iteritems():
    data = df1[(df1['filename'] == index)]
    if len(data) > 0:
        skip = int((len(data))/12)
        print("Skip value ", skip)
        for i, d in enumerate(data.index):
            if i % skip != 0:
                data.drop(d, inplace = True)
            if len(data) > 12:
                diff = 12 -len(data)
                data = data.iloc[:diff, :]
            final_csv = final_csv.append(data)
            print("Length: ", len(data))
print(len(final_csv))